# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4791, uptime 0:00:17
mini_beamline                    RUNNING   pid 4792, uptime 0:00:17
random_walk                      RUNNING   pid 4793, uptime 0:00:17
random_walk_horiz                RUNNING   pid 4794, uptime 0:00:17
random_walk_vert                 RUNNING   pid 4795, uptime 0:00:17
simple                           RUNNING   pid 4796, uptime 0:00:17
thermo_sim                       RUNNING   pid 4797, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 4798, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-18 12:26:22
Persistent Unique Scan ID: '79b569b7-bcb0-4964-9314-2136dcf32bf9'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 12:26:22.6 |     -1.000 |      99649 |
|         2 | 12:26:22.6 |     -0.500 |     101971 |
|         3 | 12:26:22.6 |      0.000 |     102730 |
|         4 | 12:26:22.6 |      0.500 |     102407 |
|         5 | 12:26:22.7 |      1.000 |     101329 |


+-----------+------------+------------+------------+
generator scan ['79b569b7'] (scan num: 1)





('79b569b7-bcb0-4964-9314-2136dcf32bf9',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/79b569b7-bcb0-4964-9314-2136dcf32bf9-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-18 12:26:23
Persistent Unique Scan ID: 'ddd6a5e0-7268-44d0-ba51-ae7f63ff075c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 12:26:23.1 |          -1 | 1128890966 |


|         2 | 12:26:23.3 |          -0 | 1131850334 |


|         3 | 12:26:23.4 |           0 | 1131369275 |


|         4 | 12:26:23.6 |           0 | 1129700447 |


|         5 | 12:26:23.7 |           1 | 1125816974 |


+-----------+------------+-------------+------------+
generator scan ['ddd6a5e0'] (scan num: 2)





('ddd6a5e0-7268-44d0-ba51-ae7f63ff075c',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 12:26:24
Persistent Unique Scan ID: 'c64da154-1b29-41d2-b2cd-388890c332d6'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 12:26:24.2 | 1101061007 |


|         2 | 12:26:25.1 | 1054186260 |


|         3 | 12:26:26.1 | 1084504180 |


...

|        28 | 12:26:51.2 | 1131365662 |


|        29 | 12:26:52.2 | 1097801232 |


|        30 | 12:26:53.2 | 1054835014 |


+-----------+------------+------------+
generator count ['c64da154'] (scan num: 3)





('c64da154-1b29-41d2-b2cd-388890c332d6',)